<a href="https://colab.research.google.com/github/dtuleva/Recipe_Recommendation_System/blob/main/tfrs_retrieval_recreate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-recommenders
# !pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 734.4 kB/s eta 0:00:00


In [36]:
from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf

# import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Recreate Recommending movies: retrieval

[tutorial link]("https://www.tensorflow.org/recommenders/examples/basic_retrieval")

Real-world recommender systems are often composed of two stages:

1. The retrieval stage is responsible for selecting an initial set of hundreds of candidates from all possible candidates. The main objective of this model is to efficiently weed out all candidates that the user is not interested in. Because the retrieval model may be dealing with millions of candidates, it has to be computationally efficient.
2. The ranking stage takes the outputs of the retrieval model and fine-tunes them to select the best possible handful of recommendations. Its task is to narrow down the set of items the user may be interested in to a shortlist of likely candidates.

In this tutorial, we're going to focus on the first stage, retrieval. If you are interested in the ranking stage, have a look at our [ranking](basic_ranking) tutorial.

Retrieval models are often composed of two sub-models:

1. A query model computing the query representation (normally a fixed-dimensionality embedding vector) using query features.
2. A candidate model computing the candidate representation (an equally-sized vector) using the candidate features

The outputs of the two models are then multiplied together to give a query-candidate affinity score, with higher scores expressing a better match between the candidate and the query.

In this tutorial, we're going to build and train such a two-tower model using the Movielens dataset.

We're going to:

1. Get our data and split it into a training and test set.
2. Implement a retrieval model.
3. Fit and evaluate it.
4. Export it for efficient serving by building an approximate nearest neighbours (ANN) index.

## The dataset

The Movielens dataset is a classic dataset from the [GroupLens](https://grouplens.org/datasets/movielens/) research group at the University of Minnesota. It contains a set of ratings given to movies by a set of users, and is a workhorse of recommender system research.

The data can be treated in two ways:

1. It can be interpreted as expressesing which movies the users watched (and rated), and which they did not. This is a form of implicit feedback, where users' watches tell us which things they prefer to see and which they'd rather not see.
2. It can also be seen as expressesing how much the users liked the movies they did watch. This is a form of explicit feedback: given that a user watched a movie, we can tell roughly how much they liked by looking at the rating they have given.

In this tutorial, we are focusing on a retrieval system: a model that predicts a set of movies from the catalogue that the user is likely to watch. Often, implicit data is more useful here, and so we are going to treat Movielens as an implicit system. This means that every movie a user watched is a positive example, and every movie they have not seen is an implicit negative example.

### Preprocess the dataset

To fit and evaluate the model, we need to split it into a training and evaluation set. In an industrial recommender system, this would most likely be done by time: the data up to time $T$ would be used to predict interactions after $T$.


In this simple example, however, let's use a random split, putting 80% of the ratings in the train set, and 20% in the test set.

In [37]:
ratings_data = pd.read_csv("/content/mock-data_interaction.csv")
recipes_data = pd.read_csv("/content/mock-data_recipe.csv")

In [38]:
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_data))
recipes = tf.data.Dataset.from_tensor_slices(dict(recipes_data))

In [39]:
# feature selection
ratings = ratings.map(lambda x: {
    "recipe_id": x["recipe_id"],
    "user_id": x["user_id"]
})

In [40]:
for r in ratings.take(1):
  print(type(r))
  for k, v in r.items():
    print(f"{k}: {v}")

<class 'dict'>
recipe_id: 222388
user_id: 8542392


In [41]:
recipes = recipes.map(lambda x: x["recipe_id"])

In [42]:
for r in recipes.take(1):
  print(r)

tf.Tensor(222388, shape=(), dtype=int64)


In [43]:
ratings_cardinality = ratings.cardinality()
ratings_cardinality

<tf.Tensor: shape=(), dtype=int64, numpy=8671>

In [44]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(ratings.cardinality(), seed=42, reshuffle_each_iteration=False)

ratings_train = shuffled.take(6_000)
ratings_test = shuffled.skip(6_000).take(2_000)

Let's also figure out unique user ids and movie titles present in the data.

This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: this allows us to look up the corresponding embeddings in our embedding tables.

In [45]:
unique_user_ids = ratings_data.user_id.unique()
len(unique_user_ids)

8266

In [46]:
unique_recipe_ids = recipes_data.recipe_id.unique()
len(unique_recipe_ids)

100

## Implementing a model

Choosing the architecture of our model is a key part of modelling.

Because we are building a two-tower retrieval model, we can build each tower separately and then combine them in the final model.

### The query tower

Let's start with the query tower.

The first step is to decide on the dimensionality of the query and candidate representations:

In [47]:
embedding_dimension = 32

Higher values will correspond to models that may be more accurate, but will also be slower to fit and more prone to overfitting.

The second is to define the model itself. Here, we're going to use Keras preprocessing layers to first convert user ids to integers, and then convert those to user embeddings via an Embedding layer. Note that we use the list of unique user ids we computed earlier as a vocabulary:

In [48]:
user_model = tf.keras.Sequential([
  tf.keras.layers.IntegerLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [53]:
user_model.layers[0](8542392)

<tf.Tensor: shape=(), dtype=int64, numpy=1>

A simple model like this corresponds exactly to a classic matrix factorization approach. While defining a subclass of tf.keras.Model for this simple model might be overkill, we can easily extend it to an arbitrarily complex model using standard Keras components, as long as we return an embedding_dimension-wide output at the end.

### The candidate tower

We can do the same with the candidate tower.

In [25]:
recipe_model = tf.keras.Sequential([
  tf.keras.layers.IntegerLookup(
      vocabulary=unique_recipe_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_recipe_ids) + 1, embedding_dimension)
])

In [55]:
recipe_model.layers[0](222388)

<tf.Tensor: shape=(), dtype=int64, numpy=1>

### Metrics

In our training data we have positive (user, movie) pairs. To figure out how good our model is, we need to compare the affinity score that the model calculates for this pair to the scores of all the other possible candidates: if the score for the positive pair is higher than for all other candidates, our model is highly accurate.

To do this, we can use the `tfrs.metrics.FactorizedTopK` metric. The metric has one required argument: the dataset of candidates that are used as implicit negatives for evaluation.

In our case, that's the `movies` dataset, converted into embeddings via our movie model:

In [26]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=recipes.batch(50).map(recipe_model)
)

In [58]:
recipes.batch(10).map(recipe_model)

<_MapDataset element_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None)>

### Loss

The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the `Retrieval` task object: a convenience wrapper that bundles together the loss function and metric computation:

In [59]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

The task itself is a Keras layer that takes the query and candidate embeddings as arguments, and returns the computed loss: we'll use that to implement the model's training loop.

### The full model

We can now put it all together into a model. TFRS exposes a base model class (`tfrs.models.Model`) which streamlines building models: all we need to do is to set up the components in the `__init__` method, and implement the `compute_loss` method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

In [60]:
class RecipeRecommendationModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.recipe_model: tf.keras.Model = recipe_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_recipe_embeddings = self.recipe_model(features["recipe_id"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_recipe_embeddings)

The tfrs.Model base class is a simply convenience class: it allows us to compute both training and test losses using the same method.

Under the hood, it's still a plain Keras model. You could achieve the same functionality by inheriting from tf.keras.Model and overriding the train_step and test_step functions (see the guide for details):

In [29]:
# class NoBaseClassMovielensModel(tf.keras.Model):

#   def __init__(self, user_model, movie_model):
#     super().__init__()
#     self.movie_model: tf.keras.Model = movie_model
#     self.user_model: tf.keras.Model = user_model
#     self.task: tf.keras.layers.Layer = task

#   def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

#     # Set up a gradient tape to record gradients.
#     with tf.GradientTape() as tape:

#       # Loss computation.
#       user_embeddings = self.user_model(features["user_id"])
#       positive_movie_embeddings = self.movie_model(features["movie_title"])
#       loss = self.task(user_embeddings, positive_movie_embeddings)

#       # Handle regularization losses as well.
#       regularization_loss = sum(self.losses)

#       total_loss = loss + regularization_loss

#     gradients = tape.gradient(total_loss, self.trainable_variables)
#     self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

#     metrics = {metric.name: metric.result() for metric in self.metrics}
#     metrics["loss"] = loss
#     metrics["regularization_loss"] = regularization_loss
#     metrics["total_loss"] = total_loss

#     return metrics

#   def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

#     # Loss computation.
#     user_embeddings = self.user_model(features["user_id"])
#     positive_movie_embeddings = self.movie_model(features["movie_title"])
#     loss = self.task(user_embeddings, positive_movie_embeddings)

#     # Handle regularization losses as well.
#     regularization_loss = sum(self.losses)

#     total_loss = loss + regularization_loss

#     metrics = {metric.name: metric.result() for metric in self.metrics}
#     metrics["loss"] = loss
#     metrics["regularization_loss"] = regularization_loss
#     metrics["total_loss"] = total_loss

#     return metrics

## Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [61]:
model = RecipeRecommendationModel(user_model, recipe_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [64]:
# cached_train = train.shuffle(100_000).batch(8192).cache()
# cached_test = test.batch(4096).cache()

cached_train = ratings_train.batch(80)
cached_test = ratings_test.batch(10)

Then train the  model:

In [65]:
model.fit(cached_train, epochs=3,)

Epoch 1/3
75/75 [==============================] - 6s 77ms/step - factorized_top_k/top_1_categorical_accuracy: 0.2030 - factorized_top_k/top_5_categorical_accuracy: 0.9192 - factorized_top_k/top_10_categorical_accuracy: 0.9440 - factorized_top_k/top_50_categorical_accuracy: 0.9953 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 182.2120 - regularization_loss: 0.0000e+00 - total_loss: 182.2120
Epoch 2/3
75/75 [==============================] - 6s 73ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1630 - factorized_top_k/top_5_categorical_accuracy: 0.9528 - factorized_top_k/top_10_categorical_accuracy: 0.9657 - factorized_top_k/top_50_categorical_accuracy: 0.9973 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 141.7542 - regularization_loss: 0.0000e+00 - total_loss: 141.7542
Epoch 3/3
75/75 [==============================] - 5s 61ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1762 - factorized_top_k/top_5_categorical_accuracy: 0.9667 - fa

If you want to monitor the training process with TensorBoard, you can add a TensorBoard callback to fit() function and then start TensorBoard using `%tensorboard --logdir logs/fit`. Please refer to [TensorBoard documentation](https://www.tensorflow.org/tensorboard/get_started) for more details.

As the model trains, the loss is falling and a set of top-k retrieval metrics is updated. These tell us whether the true positive is in the top-k retrieved items from the entire candidate set. For example, a top-5 categorical accuracy metric of 0.2 would tell us that, on average, the true positive is in the top 5 retrieved items 20% of the time.

Note that, in this example, we evaluate the metrics during training as well as evaluation. Because this can be quite slow with large candidate sets, it may be prudent to turn metric calculation off in training, and only run it in evaluation.

In [66]:
model.evaluate(cached_test, return_dict=True)

200/200 [==============================] - 13s 62ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0210 - factorized_top_k/top_5_categorical_accuracy: 0.1155 - factorized_top_k/top_10_categorical_accuracy: 0.2100 - factorized_top_k/top_50_categorical_accuracy: 0.4810 - factorized_top_k/top_100_categorical_accuracy: 0.9885 - loss: 24.3795 - regularization_loss: 0.0000e+00 - total_loss: 24.3795


{'factorized_top_k/top_1_categorical_accuracy': 0.020999999716877937,
 'factorized_top_k/top_5_categorical_accuracy': 0.11550000309944153,
 'factorized_top_k/top_10_categorical_accuracy': 0.20999999344348907,
 'factorized_top_k/top_50_categorical_accuracy': 0.48100000619888306,
 'factorized_top_k/top_100_categorical_accuracy': 0.9884999990463257,
 'loss': 22.46141242980957,
 'regularization_loss': 0,
 'total_loss': 22.46141242980957}

Test set performance is much worse than training performance. This is due to two factors:

1. Our model is likely to perform better on the data that it has seen, simply because it can memorize it. This overfitting phenomenon is especially strong when models have many parameters. It can be mediated by model regularization and use of user and movie features that help the model generalize better to unseen data.
2. The model is re-recommending some of users' already watched movies. These known-positive watches can crowd out test movies out of top K recommendations.

The second phenomenon can be tackled by excluding previously seen movies from test recommendations. This approach is relatively common in the recommender systems literature, but we don't follow it in these tutorials. If not recommending past watches is important, we should expect appropriately specified models to learn this behaviour automatically from past user history and contextual information. Additionally, it is often appropriate to recommend the same item multiple times (say, an evergreen TV series or a regularly purchased item).